In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime, timedelta

def rsi(series, period=14):
    """Calculate traditional RSI"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def streak_rsi(series, period=2):
    """Calculate RSI of price streaks"""
    # Calculate streaks
    price_change = series.diff()
    streaks = []
    current_streak = 0
    
    for change in price_change:
        if pd.isna(change):
            streaks.append(0)
            current_streak = 0
        elif change > 0:
            if current_streak >= 0:
                current_streak += 1
            else:
                current_streak = 1
            streaks.append(current_streak)
        elif change < 0:
            if current_streak <= 0:
                current_streak -= 1
            else:
                current_streak = -1
            streaks.append(current_streak)
        else:  # change == 0
            streaks.append(0)
            current_streak = 0
    
    streak_series = pd.Series(streaks, index=series.index)
    return rsi(streak_series, period)

def percent_rank(series, period=100):
    """Calculate percent rank over a rolling window"""
    def rank_pct(x):
        if len(x) < 2:
            return 50.0
        current_value = x.iloc[-1]
        rank = (x < current_value).sum()
        return (rank / (len(x) - 1)) * 100
    
    return series.rolling(window=period).apply(rank_pct, raw=False)

def connors_rsi(data, rsi_period=3, streak_period=2, rank_period=100):
    """
    Calculate Connors RSI (CRSI)
    
    CRSI = (RSI(Close, 3) + RSI(Streak, 2) + PercentRank(ROC, 100)) / 3
    
    Parameters:
    - rsi_period: Period for price RSI (default 3)
    - streak_period: Period for streak RSI (default 2)
    - rank_period: Period for percent rank (default 100)
    """
    close = data['Close']
    
    # Component 1: RSI of closing prices
    price_rsi = rsi(close, rsi_period)
    
    # Component 2: RSI of price streaks
    streak_rsi_values = streak_rsi(close, streak_period)
    
    # Component 3: Percent rank of rate of change
    roc = close.pct_change() * 100  # Rate of change as percentage
    percent_rank_values = percent_rank(roc, rank_period)
    
    # Calculate Connors RSI
    crsi = (price_rsi + streak_rsi_values + percent_rank_values) / 3
    
    return {
        'CRSI': crsi,
        'Price_RSI': price_rsi,
        'Streak_RSI': streak_rsi_values,
        'Percent_Rank': percent_rank_values
    }

def generate_signals(crsi, oversold=20, overbought=80):
    """Generate trading signals based on CRSI levels"""
    signals = pd.Series(index=crsi.index, dtype='object')
    
    # Buy signals when CRSI crosses above oversold level
    buy_condition = (crsi <= oversold) & (crsi.shift(1) > oversold)
    signals[buy_condition] = 'BUY'
    
    # Sell signals when CRSI crosses below overbought level  
    sell_condition = (crsi >= overbought) & (crsi.shift(1) < overbought)
    signals[sell_condition] = 'SELL'
    
    return signals

def plot_connors_rsi(data, crsi_data, signals, symbol='Stock'):
    """Create interactive Plotly chart with price and Connors RSI"""
    
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=[f'{symbol} Price Chart', 'Connors RSI Components', 'Connors RSI with Signals'],
        row_width=[0.3, 0.3, 0.4]
    )
    
    # Price chart with candlesticks
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            name='Price',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Add buy/sell signals on price chart
    buy_signals = signals[signals == 'BUY']
    sell_signals = signals[signals == 'SELL']
    
    if not buy_signals.empty:
        fig.add_trace(
            go.Scatter(
                x=buy_signals.index,
                y=data.loc[buy_signals.index, 'Close'],
                mode='markers',
                marker=dict(symbol='triangle-up', size=12, color='green'),
                name='Buy Signal',
                showlegend=True
            ),
            row=1, col=1
        )
    
    if not sell_signals.empty:
        fig.add_trace(
            go.Scatter(
                x=sell_signals.index,
                y=data.loc[sell_signals.index, 'Close'],
                mode='markers',
                marker=dict(symbol='triangle-down', size=12, color='red'),
                name='Sell Signal',
                showlegend=True
            ),
            row=1, col=1
        )
    
    # CRSI Components
    fig.add_trace(
        go.Scatter(
            x=crsi_data['Price_RSI'].index,
            y=crsi_data['Price_RSI'],
            name='Price RSI',
            line=dict(color='blue', width=1)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=crsi_data['Streak_RSI'].index,
            y=crsi_data['Streak_RSI'],
            name='Streak RSI',
            line=dict(color='orange', width=1)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=crsi_data['Percent_Rank'].index,
            y=crsi_data['Percent_Rank'],
            name='Percent Rank',
            line=dict(color='purple', width=1)
        ),
        row=2, col=1
    )
    
    # Main Connors RSI
    fig.add_trace(
        go.Scatter(
            x=crsi_data['CRSI'].index,
            y=crsi_data['CRSI'],
            name='Connors RSI',
            line=dict(color='black', width=2)
        ),
        row=3, col=1
    )
    
    # Add horizontal lines for overbought/oversold levels
    fig.add_hline(y=80, line_dash="dash", line_color="red", row=3, col=1)
    fig.add_hline(y=20, line_dash="dash", line_color="green", row=3, col=1)
    fig.add_hline(y=50, line_dash="dot", line_color="gray", row=3, col=1)
    
    # Add the same lines to components chart
    fig.add_hline(y=80, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=20, line_dash="dash", line_color="green", row=2, col=1)
    fig.add_hline(y=50, line_dash="dot", line_color="gray", row=2, col=1)
    
    # Update layout
    fig.update_layout(
        title=f'{symbol} - Connors RSI Analysis',
        height=800,
        showlegend=True,
        xaxis_rangeslider_visible=False
    )
    
    # Update y-axes
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="RSI Components", row=2, col=1, range=[0, 100])
    fig.update_yaxes(title_text="Connors RSI", row=3, col=1, range=[0, 100])
    fig.update_xaxes(title_text="Date", row=3, col=1)
    
    return fig



In [ ]:
# Download sample data (Apple stock for last 2 years)
symbol = "TLSA"
end_date = datetime.now()
start_date = end_date - timedelta(days=730)

In [ ]:
print(f"Downloading {symbol} data...")
data = yf.download(symbol, start=start_date, end=end_date,multi_level_index=False)

In [ ]:
data.head()

In [ ]:
print("Calculating Connors RSI...")
    
# Calculate Connors RSI
crsi_data = connors_rsi(data)

# Generate trading signals
signals = generate_signals(crsi_data['CRSI'])

# Print some statistics
print(f"\nConnors RSI Statistics for {symbol}:")
print(f"Current CRSI: {crsi_data['CRSI'].iloc[-1]:.2f}")
print(f"Average CRSI: {crsi_data['CRSI'].mean():.2f}")
print(f"CRSI Standard Deviation: {crsi_data['CRSI'].std():.2f}")

In [ ]:
# Count signals
buy_count = (signals == 'BUY').sum()
sell_count = (signals == 'SELL').sum()
print(f"\nTrading Signals Generated:")
print(f"Buy signals: {buy_count}")
print(f"Sell signals: {sell_count}")

In [ ]:
# Create and show plot
print("\nGenerating interactive chart...")
fig = plot_connors_rsi(data, crsi_data, signals, symbol)
fig.show()

In [ ]:

# Show recent signals
recent_signals = signals.dropna().tail(10)
if not recent_signals.empty:
    print(f"\nRecent Signals:")
    for date, signal in recent_signals.items():
        price = data.loc[date, 'Close']
        print(f"{date.strftime('%Y-%m-%d')}: {signal} at ${price:.2f}")

